# MD simulatuion of Ni/TiO2 interfaces with PFP

* MD trajectories will be used for the valiation of LightPFP models

In [ ]:
model_version="v7.0.0"
calc_mode="crystal_u0"

* Define the MD script

In [ ]:
import numpy as np
from time import perf_counter
from ase import units
from ase.io import Trajectory
from ase.md.langevin import Langevin
from ase.md.nvtberendsen import NVTBerendsen
from ase.md.npt import NPT
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from light_pfp_data.utils.atoms_utils import convert_atoms_to_upper


class PrintDyn:
    def __init__(self, dyn, logfile=None):
        self.dyn = dyn
        self.st = perf_counter()
        self.logfile = logfile
        if self.logfile is not None:
            with open(self.logfile, "w") as fd:
                fd.write("# step E_tot E_pot density T elapsed_time\n")
    def __call__(self):
        dyn = self.dyn
        atoms = dyn.atoms
        msg = (
            f"{dyn.get_number_of_steps(): 6d} {atoms.get_total_energy():.3f} {atoms.get_potential_energy():.3f} "
            f"{atoms.get_masses().sum() / units.kg / atoms.get_volume() * 1e27:.5f} "
            f"{atoms.get_temperature():.1f} {perf_counter() - self.st:.2f}"
        )
        print(msg)
        if self.logfile is not None:
            with open(self.logfile, "a") as fd:
                fd.write(msg+"\n")
                

def md_protocol(atoms, temperature, steps_nvt, steps_npt, traj):
    """
    Define a MD script
    1. NVT MD
    2. NPT MD
    """
    traj = Trajectory(traj, "w", atoms=atoms)
    # Initialize atomic velocities according to Maxwell-Boltzmann distribution.
    MaxwellBoltzmannDistribution(atoms, temperature_K=temperature)
    
    # Run a short NVT MD simulation (e.g., using Langevin dynamics) to equilibrate the structure.
    md = Langevin(atoms, 1.0 * units.fs, temperature_K=temperature, friction=0.1)
    print_info = PrintDyn(md)
    md.attach(print_info, interval=100)
    md.attach(traj.write, interval=100)
    md.run(steps=steps_nvt)
    
    # Then run a longer NPT MD simulation to generate diverse training structures.
    md = NPT(
        atoms,
        1.0 * units.fs,
        temperature_K=temperature,
        externalstress=1.0 * units.bar,
        mask=np.eye(3),
        ttime=20.0 * units.fs,
        pfactor=2e6 * units.GPa * (units.fs**2)
    )
    print_info = PrintDyn(md)
    md.attach(print_info, interval=100)
    md.attach(traj.write, interval=100)
    md.run(steps=steps_npt)


In [ ]:
from ase.io import read
from pfp_api_client import ASECalculator, Estimator

def md_wrap(task, temperature):
    atoms = read(f"inputs/{task}.xyz")
    atoms = convert_atoms_to_upper(atoms)
    calc = ASECalculator(Estimator(model_version=model_version, calc_mode=calc_mode))
    atoms.calc = calc
    md_protocol(atoms, temperature, 5000, 100000, f"pfp_md/pfp_md_{task}_{temperature}.traj")


* Run MD tasks in parallel with joblib module

In [ ]:
from pathlib import Path
from joblib.parallel import Parallel, delayed

Path("pfp_md").mkdir(exist_ok=True)

Parallel(n_jobs=3)(delayed(md_wrap)(task, temp) for task in ["anatase_TiO2_Ni_0", "anatase_TiO2_Ni_1", "anatase_TiO2_Ni_2", "anatase_TiO2_Ni_3", "anatase_TiO2_Ni_4", "anatase_TiO2_Ni_5"] for temp in [1200, 1400, 1600])